In [1]:
import argparse
from transformers import AutoTokenizer
import torch
import os
from pointllm.conversation import conv_templates, SeparatorStyle
from pointllm.utils import disable_torch_init
from pointllm.model import *
from pointllm.model.utils import KeywordsStoppingCriteria

from pointllm.data import load_ulip2_objaverse_point_cloud
import copy
import os
from pointllm.model.pointbert.seg_xcoders import *
from easydict import EasyDict
import yaml
import torch.nn as nn
from pointllm.model.pointbert.pointnet2_utils import PointNetFeaturePropagation
from pointnet2_ops import pointnet2_utils
from pointllm.model.pointbert.transformer  import TwoWayTransformer
from torch.nn import functional as F
from pointllm.model.pointbert.point_segmentor import PointSegTransformer
from torch.utils.data import DataLoader

In [2]:
def cfg_from_yaml_file(cfg_file):
    config = EasyDict()
    with open(cfg_file, 'r') as f:
        new_config = yaml.load(f, Loader=yaml.FullLoader)
    merge_new_config(config=config, new_config=new_config)
    return config
def merge_new_config(config, new_config):
    for key, val in new_config.items():
        if not isinstance(val, dict):
            if key == '_base_':
                with open(new_config['_base_'], 'r') as f:
                    try:
                        val = yaml.load(f, Loader=yaml.FullLoader)
                    except:
                        val = yaml.load(f)
                config[key] = EasyDict()
                merge_new_config(config[key], val)
            else:
                config[key] = val
                continue
        if key not in config:
            config[key] = EasyDict()
        merge_new_config(config[key], val)
    return config
def fps(data, number):
    '''
        data B N 3
        number int
    '''
    fps_idx = pointnet2_utils.furthest_point_sample(data, number) 
    fps_data = pointnet2_utils.gather_operation(data.transpose(1, 2).contiguous(), fps_idx).transpose(1,2).contiguous()
    return fps_data

In [3]:
point_bert_config_name = "./pointllm/model/pointbert/PointTransformer_base_8192point.yaml"
point_bert_config = cfg_from_yaml_file(point_bert_config_name)
point_bert_config.model.point_dims = 6
use_max_pool = getattr(point_bert_config.model, "use_max_pool", False)

In [4]:
from pointllm.model.pointbert.part_prompt_dataset import *

In [5]:
dataset = PartPromptDataset(point_bert_config)
dataloader= DataLoader(dataset,10)

In [6]:
for i in dataloader:
    data = i
    break

In [9]:
data[0].shape

torch.Size([10, 8192, 6])

In [ ]:
checkpoint = torch.load("/data1/linfeng/PartLLM/experiment-5/model_34000.pth", map_location=lambda storage, loc: storage.cuda())
segmentor = PointSegTransformer(point_bert_config,use_max_pool).cuda()
# segmentor.prompt_encoder.load_state_dict(checkpoint["prompt_encoder"])
# segmentor.mask_decoder.load_state_dict(checkpoint["mask_decoder"])

In [ ]:
point,text_embedding, label,_ = data
point = point.cuda()
text_embedding = text_embedding.cuda()
mask,iou = segmentor(point,text_embedding)
mask = mask>0

In [ ]:
mask.shape

torch.Size([10, 8192])

In [ ]:
import pyviz3d.visualizer as viz
ind = 5
v = viz.Visualizer()
points = point[ind].cpu().numpy()
color1 = np.zeros((points.shape[0],3))
color1[label[ind]==1] = [255.0,0,0]
v.add_points("label", points[:,:3], color1, point_size=10)
color2 = np.zeros((points.shape[0],3))
color2[mask[ind].cpu()==True] = [0,255,0]
v.add_points("predict", points[:,:3], color2, point_size=10)
v.save('./parnet_vis/example_point_clouds')


************************************************************************
1) Start local server:
    cd /home/linfeng/PartLLM/parnet_vis/example_point_clouds; python -m http.server 6008
2) Open in browser:
    http://0.0.0.0:6008
************************************************************************


In [ ]:
mask

tensor([[False, False, False,  ..., False, False, False]], device='cuda:0')

In [ ]:
3/4

0.75

In [ ]:
point,text_embedding,label = dataset[0]

In [ ]:
point

NameError: name 'point' is not defined

In [ ]:
segmentor = PointSegTransformer(point_bert_config,use_max_pool).cuda()

In [ ]:
masks,iou_pred = segmentor(data[0].cuda(),data[1].cuda())

In [ ]:
masks.shape

torch.Size([10, 8192])

In [ ]:
iou_pred.shape

torch.Size([10, 1])

In [ ]:
data[2].shape

torch.Size([10, 8192])

In [ ]:
point_backbone = PointEncoder(point_bert_config.model, use_max_pool=use_max_pool).cuda()

In [ ]:
input = torch.Tensor(3,8192,6).cuda()
# for i in range(input.shape[0]):
#     input[i] = pc_norm(input[i])

In [ ]:
ouput = point_backbone(input)[:,1:,:]

In [ ]:
prompt_encoder = PromptEncoder(point_bert_config).cuda()

In [ ]:
input2 = torch.Tensor(3,512).cuda().unsqueeze(dim=1)

In [ ]:
input2.shape

torch.Size([3, 1, 512])

In [ ]:
out = prompt_encoder(input2)

In [ ]:
point_backbone.pts.shape

torch.Size([3, 512, 3])

In [ ]:
mask_decoder = MaskDecoder(point_bert_config).cuda()

In [ ]:
for i in range(ouput.shape[0]):
    x = mask_decoder(point_backbone.pts[i:i+1],input[i:i+1,:,:3],ouput[i:i+1],out[i:i+1])
    break

In [ ]:
pts = torch.Tensor(3,8192,6).cuda()
prompts = torch.Tensor(3,1,512).cuda()

point_embedding = point_backbone(pts)[:,1:,:]
prompt_embedding = prompt_encoder(prompts)
masks,iou_pred = mask_decoder(point_backbone.pts,pts[:,:,:3],point_embedding,prompt_embedding)

NameError: name 'prompt_encoder' is not defined

In [ ]:
class MLP(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        output_dim: int,
        num_layers: int,
        sigmoid_output: bool = False,
    ) -> None:
        super().__init__()
        self.num_layers = num_layers
        h = [hidden_dim] * (num_layers - 1)
        self.layers = nn.ModuleList(
            nn.Linear(n, k) for n, k in zip([input_dim] + h, h + [output_dim])
        )
        self.sigmoid_output = sigmoid_output

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = F.relu(layer(x)) if i < self.num_layers - 1 else layer(x)
        if self.sigmoid_output:
            x = F.sigmoid(x)
        return x

In [ ]:
from torch import nn
import torch

class PointEncoder(nn.Module):
    def __init__(self, config, **kwargs):
        super().__init__()
        
        self.model = torch.load(config.point_encoder_path)

    def forward(input):
        return self.model(input)

In [ ]:
class PromptEncoder(nn.Module):
    def __init__(self, config, **kwargs):
        super().__init__()

        self.embed_dim = config.out_dim

        self.no_mask_embed = nn.Embedding(1, self.embed_dim)

    def _get_batch_size(self,prompt):
        if prompt is not None:
            return prompt.shape[0]
        else:
            return 1

    def forward(self, prompt, **kwargs):
        bs = self._get_batch_size(prompt)
        sparse_embeddings = torch.empty((bs,0,self.embed_dim), device = self.no_mask_embed.weight.device)

        if prompt is not None:
            sparse_embeddings = torch.cat([sparse_embeddings, prompt], dim = 1)

        return sparse_embeddings

In [ ]:
slice(1,None)

slice(1, None, None)

In [ ]:
slice(0,1)

slice(0, 1, None)

In [ ]:
import numpy as np
x = np.random.randn(2,4,10,20)

In [ ]:
def init_model(args):
    # Model
    disable_torch_init()

    model_path = args.model_path 
    print(f'[INFO] Model name: {model_path}')

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = PointLLMLlamaForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=False, use_cache=True, torch_dtype=torch.float16).cuda()
                                                     # args.torch_dtype).cuda()
    model.initialize_tokenizer_point_backbone_config_wo_embedding(tokenizer)

    model.eval()

    mm_use_point_start_end = getattr(model.config, "mm_use_point_start_end", False)
    # Add special tokens ind to model.point_config
    point_backbone_config = model.get_model().point_backbone_config
    
    if mm_use_point_start_end:
        if "v1" in model_path.lower():
            conv_mode = "vicuna_v1_1"
        else:
            raise NotImplementedError

        conv = conv_templates[conv_mode].copy()

    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    
    return model, tokenizer, point_backbone_config, keywords, mm_use_point_start_end, conv

In [ ]:
import numpy as np
def pc_norm(pc):
    """ pc: NxC, return NxC """
    xyz = pc[:, :3]
    other_feature = pc[:, 3:]

    centroid = np.mean(xyz, axis=0)
    xyz = xyz - centroid
    m = np.max(np.sqrt(np.sum(xyz ** 2, axis=1)))
    xyz = xyz / m

    pc = np.concatenate((xyz, other_feature), axis=1)
    return pc
def load_ulip2_objaverse_point_cloud(data_path, object_id, pointnum=8192, use_color=False):

    if not use_color:
        filename = f"{object_id}/{object_id}_{pointnum}.npz"
        point_cloud = np.load(os.path.join(data_path, filename))['arr_0'] # * pointnum, 3 array
    else:
        filename = f"{object_id}_{pointnum}.npy"
        point_cloud = np.load(os.path.join(data_path, filename))

    # * normalize
    point_cloud = pc_norm(point_cloud)

    return point_cloud

def load_my_own_point_cloud(object_id):
    # /data2/llf/fss_data/ScanNet/scenes/data/scene0000_00.npy
    point_cloud = np.load(object_id)
    point_cloud = point_cloud[:,:6]
    point_cloud[:,3:] = point_cloud[:,3:]/255

    point_cloud = pc_norm(point_cloud)

    return torch.from_numpy(point_cloud).unsqueeze_(0).to(torch.float32)
    

def load_point_cloud(args):
    object_id = args.object_id
    print(f"[INFO] Loading point clouds using object_id: {object_id}")
    point_cloud = load_ulip2_objaverse_point_cloud(args.data_path, object_id, pointnum=8192, use_color=True)
    
    return object_id, torch.from_numpy(point_cloud).unsqueeze_(0).to(torch.float32)

In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, \
default="RunsenXu/PointLLM_7B_v1.1")

parser.add_argument("--data-path", type=str, default="/data2/llf/objaverse/8192_npy")
parser.add_argument("--torch-dtype", type=str, default="float32", choices=["float32", "float16", "bfloat16"])
args = parser.parse_args(args=[])
    # '--torch-dtype','float32'])

In [ ]:
model, tokenizer, point_backbone_config, keywords, mm_use_point_start_end, conv = init_model(args)

[INFO] Model name: RunsenXu/PointLLM_7B_v1.1
Loading PointBERT config from /home/linfeng/PartLLM/pointllm/model/pointbert/PointTransformer_base_8192point.yaml.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
point_encoder =copy.deepcopy(model.model.point_backbone)

In [ ]:
torch.save(point_encoder,"point_encoder_7B.pth")

In [ ]:
tokenizer("body of chair")

{'input_ids': [1, 3573, 310, 11774], 'attention_mask': [1, 1, 1, 1]}

In [ ]:
tokenizer

LlamaTokenizer(name_or_path='RunsenXu/PointLLM_7B_v1.1', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<unk>'})

In [ ]:
voc_embed = copy.deepcopy(model.model.embed_tokens)

In [ ]:
torch.save(voc_embed,"voc_embed_7B.pth")

In [ ]:
voc_embed(tokenizer("body of chair")['input_ids'])

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list

In [ ]:
from transformers import CLIPProcessor, CLIPModel

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
inputs = processor(text=["a photo of a cat", "a photo of a dog"], return_tensors="pt", padding=True)

In [ ]:
inputs

{'input_ids': tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
text_features = model.get_text_features(**inputs)

In [ ]:
text_features

torch.Size([2, 512])

In [ ]:
import json
with open('./parnet_vis/meta_class.json', 'r') as fcc_file:
    meta_class = json.load(fcc_file)
meta_class = [" ".join(x.split("_")) for x in meta_class]
inputs = processor(text=meta_class, return_tensors="pt", padding=True)
text_features = model.get_text_features(**inputs)

In [ ]:
torch.save(text_features,"./text_features.pth")

In [ ]:
torch.rand(1,8192,3).shape

torch.Size([1, 8192, 3])

In [ ]:
point_encoder = point_encoder.to(torch.float32)
point_feature = point_encoder(torch.rand(1,8192,6).to(torch.float32).cuda())

In [ ]:
point_feature.shape

torch.Size([1, 513, 1152])

In [ ]:
ln = torch.nn.LayerNorm(50)

In [ ]:
ln.weight.shape

torch.Size([50])

In [ ]:
bn = torch.nn.BatchNorm2d(50)

In [ ]:
bn.weight.shape

torch.Size([50])

In [ ]:
a= torch.normal(0,1,size=(8,100)).sigmoid()
b=(torch.normal(0,1,size=(8,100))>0).long()

In [ ]:
inputs = a
targets = b
scale = 1000
eps = 1e-6
numerator = 2 * (inputs / scale * targets).sum(-1)
denominator = (inputs / scale).sum(-1) + (targets / scale).sum(-1)
loss = 1 - (numerator + eps) / (denominator + eps)


In [ ]:
loss

tensor([0.5685, 0.5047, 0.4492, 0.4678, 0.4971, 0.4818, 0.5235, 0.5119])

In [ ]:
c= (a>0.5).long()
(c*targets).sum(-1)

tensor([17, 25, 30, 28, 24, 28, 27, 20])

In [ ]:
c.sum(-1)

tensor([46, 51, 52, 48, 44, 52, 59, 40])

In [ ]:
d = (c*targets).sum(-1)/torch.sum(c,-1)

In [ ]:
d.unsqueeze(1).shape

torch.Size([8, 1])

In [ ]:
text = torch.load("/data2/llf/partnet/text_embeddings.pth").detach()

In [ ]:
torch.save(text,"/data2/llf/partnet/text_embeddings.pth")